In [1]:
import os
import PyPDF2

def extract_text_from_pdfs(pdf_folder):
    all_text = ''
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            path = os.path.join(pdf_folder, filename)
            with open(path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    all_text += page.extract_text() + '\n'
    return all_text

pdf_folder = './pdfs'  # altere para o caminho real
corpus_text = extract_text_from_pdfs(pdf_folder)


In [4]:
import re
import nltk
from nltk.tokenize import sent_tokenize

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'[^a-záéíóúàãõç\s]', '', text)
    return text

nltk.data.path.append("./nltk_data")
processed_text = preprocess_text(corpus_text)
sentences = sent_tokenize(processed_text, language='portuguese')


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/portuguese/[0m

  Searched in:
    - 'C:\\Users\\carlo/nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\carlo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - './nlkt_data'
    - './nltk_data'
    - './nltk_data'
**********************************************************************


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Entrada da pergunta
input_seq = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], trainable=False)(input_seq)
lstm = LSTM(128)(embedding_layer)
output = Dense(num_classes, activation='softmax')(lstm)

model = Model(inputs=input_seq, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


'Mercado Ver-o-Peso\nCultura e turismo  de Belém (Pará)\nCultura e turismo de Belém  são diversificadas as opções culturais e turísticas da capital do\nEstado do pará , influenciadas por indígenas e imigrantes estrangeiros, que manifestam-se através\ndas manifestações religiosas, da gastronomia, do folclore, danças, músicas, teatros, museus, etc. A\ncapital paraense desponta como grande roteiro turístico do Brasil, gerando uma excelente\noportunidade para investimentos turísticos.\nBioparque Amazônia  (1989 ): parque particular\ncriado pelo médico Jorge Aarão\nMonteiro.[1][2][3][4][5]\nBondinho de Belém: tecnologia implantada na\népoca da Belle Époque  (1871-1914).\nCasa das Onze Janelas  (século XVIII ,\naproximadamente 1750 ): vizinho do Forte do\nCastelo, feito pelo dono de engenho de açúcar  Domingos da Costa Bacelar .[6][7][8]\nComplexo Feliz Lusitânia  (1616 ): um povoado colonial português  criado pelo capitão Francisco\nCaldeira Castelo Branco  às margens da baía do Guajará. Nu